# Deploy and verify the deployment

Steps:
- Deploy model using docker image
- Run inference to check if its up and running

In [1]:
!pip install kfp --upgrade -q

In [2]:
import kfp
from kfp import dsl

In [3]:
client = kfp.Client()

## Deploy Model

In [47]:
def deploy(image_url:str, deployment_name:str, predictor_name:str):
    import subprocess
    
    with open("/tmp/manifest.yaml", "w") as f:
        manifest = f"""
apiVersion: machinelearning.seldon.io/v1
kind: SeldonDeployment
metadata:
  name: {deployment_name}
spec:
  name: {deployment_name}
  predictors:
  - componentSpecs:
    - spec:
        containers:
        - name: classifier
          image: {image_url}
          env:
          - name: HF_DATASETS_OFFLINE
            value: "1"
          - name: TRANSFORMERS_OFFLINE
            value: "1"
          securityContext:
            allowPrivilegeEscalation: false
            runAsUser: 0
          resources:
            limits:
              nvidia.com/gpu: 1
    graph:
      name: classifier
    name: {predictor_name}
    replicas: 1
    labels:
      sidecar.istio.io/inject: "false"
        """
        print(manifest)
        f.write(manifest)
    
    result = subprocess.call(['kubectl', 'apply', '-f', '/tmp/manifest.yaml', '-n', 'admin'])
    assert result == 0

In [48]:
deploy_op = kfp.components.create_component_from_func(
        func=deploy,
        output_component_file='deploy-component.yaml', # This is optional. It saves the component spec for future use.
        base_image='bponieckiklotz/seldon-deploy:0.1',
        packages_to_install=[])

## Check response

In [49]:
def validate(deployment_name:str, predictor_name:str, namespace:str, max_retries:int=20, sleep_time:int=30):
    import base64
    import json
    import logging
    import time

    import requests
    from PIL import Image
    url = "http://images.cocodataset.org/val2017/000000039769.jpg"
    cat_image = Image.open(requests.get(url, stream=True).raw)
    files = {
        "mode": cat_image.mode,
        "size": "640x480",
        "media": base64.b64encode(cat_image.tobytes())
    }
    
    deployed = False
    count = max_retries
    while not deployed:
        try:
            res = requests.post(f"http://{deployment_name}-{predictor_name}.{namespace}.svc.cluster.local:8000/api/v0.1/predictions", files=files)
            print(f"Result code {res.status_code}")
            boxes = json.loads(res.text)
            for b in boxes:
                if b['score'] > 0.9:
                    print(f"Detected {b['label']} with confidence {round(b['score'], 3)} at location {b['box']}")
            if res.status_code == 200:
                deployed = True
            else:
                count -= 1
                print("Sleeping 1 sec")
                time.sleep(sleep_time)
        except Exception as e:
            print(e)
            count -= 1
            print(f"Sleeping {sleep_time} sec. (except)")
            time.sleep(sleep_time)


In [51]:
validate_op = kfp.components.create_component_from_func(
        func=validate,
        output_component_file='validate-component.yaml', # This is optional. It saves the component spec for future use.
        base_image='bponieckiklotz/seldon-deploy:0.1',
        packages_to_install=['requests', 'Pillow'])

## Create pipeline

In [52]:
from kubernetes.client.models import V1EnvVar
from kfp.onprem import use_k8s_secret

@dsl.pipeline(
    name="object_detection_pipeline",
    description="Object Detection pipeline",
)
def object_detection_deploy_pipeline(image_url, deployment_name="object-detection", predictor_name="default", namespace="admin"):
    deploy_task = deploy_op(image_url=image_url, 
                            deployment_name=deployment_name, 
                            predictor_name=predictor_name)
    validate_task = validate_op(deployment_name=deployment_name, 
                                predictor_name=predictor_name, 
                                namespace=namespace
                               ).after(deploy_task)


In [53]:
client.create_run_from_pipeline_func(
    object_detection_deploy_pipeline,
    arguments={
        "image_url": "bponieckiklotz/jellyfish.object-detection:dev@sha256:b19fb9d48e43e1c0a9afe6480553ae506b198d48e347be3f13f9d794f0b5e270",
        "deployment_name": "object-detection",
        "predictor_name": "default",
        "namespace": "admin",
    })

RunPipelineResult(run_id=8517121c-f0d8-4d00-8c79-49119743fb85)